In [1]:
import pickle
import pandas as pd
import matplotlib
matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'],'size':8})
matplotlib.rcParams['pdf.fonttype'] = 42


/Users/sofiepalmuskronborg/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
# load data
file_in = open("all_ads_labeled.pickle", "rb")
dat = pickle.load(file_in)

print("number of ads", dat.shape)

ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'

In [3]:
# convert to CSV format
data = pd.read_csv(
    "/Users/sofiepalmuskronborg/Library/CloudStorage/OneDrive-ITU/ITU/Thesis/ad_data.csv"
)

In [4]:
print("number of ads", data.shape)

number of ads (602544, 17)


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602544 entries, 0 to 602543
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ad_creation_time          602544 non-null  object 
 1   ad_creative_body          598057 non-null  object 
 2   spend                     602544 non-null  float64
 3   impressions               602544 non-null  float64
 4   delivery_by_region        463401 non-null  object 
 5   demographic_distribution  463401 non-null  object 
 6   page_id                   602544 non-null  int64  
 7   page_name                 602544 non-null  object 
 8   bylines                   600813 non-null  object 
 9   id                        602544 non-null  int64  
 10  spend_lo                  602544 non-null  int64  
 11  spend_hi                  602544 non-null  int64  
 12  impressions_lo            602544 non-null  int64  
 13  impressions_hi            602544 non-null  i

In [6]:
data.isnull().sum()
# demographic distribution of ads

ad_creation_time                 0
ad_creative_body              4487
spend                            0
impressions                      0
delivery_by_region          139143
demographic_distribution    139143
page_id                          0
page_name                        0
bylines                       1731
id                               0
spend_lo                         0
spend_hi                         0
impressions_lo                   0
impressions_hi                   0
label                            0
party                            0
state                            0
dtype: int64

In [7]:
data.nunique()

ad_creation_time              1237
ad_creative_body             58449
spend                           42
impressions                     39
delivery_by_region          335516
demographic_distribution    459236
page_id                        518
page_name                      553
bylines                        955
id                          459728
spend_lo                        42
spend_hi                        42
impressions_lo                  39
impressions_hi                  39
party                            2
state                           54
dtype: int64

## Cleaning the data

In [17]:
# Removing nans and empty strings (ad_creative_body, demographic_distribution, delivery_by_region)

# filter out nans in creative sentiment (e.g. they are probably videos), removes 4487 ads (0.74%)
data = data[
    ~data["ad_creative_body"].isna()
]  # -> politician Lois Frankel with id: 145862215492466 is gone from the dataset with 2 ads

# filter out ads which just have empty space as text, e.g '  '(no 'real' text content), removes 40 in total out of (602544) total 5527 ads ^ for ad_creative_body (0.75%)
data = data[data['ad_creative_body'].str.strip() != '']

# filter out ads that do not have any targeting info (removes 139,143 ads - 23,09% of all ads) - see analysis below
data = data[
    ~((data["demographic_distribution"].isna()) | (data["delivery_by_region"].isna()))
]  # -> Senator Rob Portman with id: 45243961073" is gone from the dataset with 1 ad (and no demographics)

# remove ads where demographic_distribution = [{'percentage': '1'}], not sure what that means, removes 6 ads
data = data[data['demographic_distribution'].map(lambda x: 1 if x == [{'percentage': '1'}] else 0) == 0]

In [18]:
# drop the column 'label' marking climate/non climate adds
data = data.drop('label', axis=1)

# relabel Independents as Democrats - as the two independent politicians caucus with the Democrats
data["party"].replace("Independent", "Democrat", inplace=True)

/var/folders/cr/2b4bjd0d6cscnr8h_66vmrqm0000gn/T/ipykernel_1888/638186792.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["party"].replace("Independent", "Democrat", inplace=True)


In [12]:
print("Number of ads after basic cleaning", data.shape)

Number of ads after basic cleaning (459728, 16)


In [14]:
data.to_csv('filtered_data.csv')